In [1]:
import cv2
import dill
import pandas as pd
import numpy as np

In [2]:
imgFreq = {}
fList = ['19H','19V','22V','37V','37H','91H','91V','150H','183_1H','183_3H','183_7H']
rList = ['ATL','CPAC','EPAC','IO','SHEM','WPAC']
for r in rList:
    imgFreq[r] = {}
    for f in fList:
        imgFreq[r][f] = pd.read_csv("..//ImagesPerFreq//"+r+"_"+f+'.csv')
        del imgFreq[r][f]['Unnamed: 0']

For Testing purpose considering:-<br>
ATL region | 19H freq | 90+ Area<br>
ATL region | 183_1H freq | 90+ Area

In [3]:
atl_19h = imgFreq['ATL']['19H'].loc[imgFreq['ATL']['19H']['Area'] >= 0.90]
atl_183_1h = imgFreq['ATL']['183_1H'].loc[imgFreq['ATL']['183_1H']['Area'] >= 0.90]

In [6]:
with open('atl_19h_matrix.pkl', 'rb') as file:
    matOfImages = dill.load(file)

with open('atl_19h_cluster.pkl', 'rb') as file:
    kmeans = dill.load(file)

#Read ImageFiles
matOfImages = np.zeros( (0,129602) ).astype(np.uint8)
for index, row in atl_183_1h.iterrows():
    imgName = row[0]
    img_cv = cv2.imread(imgName,cv2.IMREAD_UNCHANGED)  
    img_cv = cv2.resize(img_cv, (360,360))
    gray_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY).flatten()
    alldata = np.append(gray_cv, [row[3],row[4]], 0)
    matOfImages = np.append( matOfImages, [alldata], axis = 0 )
    
    print(len(matOfImages))

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=7, random_state=0).fit(matOfImages)

In [9]:
matOfImages

array([[255, 255, 255, ..., 255, 983,  50],
       [255, 255, 255, ..., 255, 982,  55],
       [255, 255, 191, ..., 255, 980,  60],
       ...,
       [255, 255, 255, ..., 255, 997,  55],
       [255, 255, 255, ..., 255, 994,  55],
       [255, 255, 255, ..., 255, 994,  55]])

In [ ]:
kmeans

In [7]:
clusters = kmeans.predict(  )

In [ ]:
import shutil
for i in range(len(clusters)):
    shutil.copy(atl_183_1h.iloc[i,0], '..\\..\\AllFrequencies\\ATL\\183_1H\\'+str(clusters[i]) )

In [ ]:
import dill
with open('atl_183_1h_matrix.pkl', 'wb') as file:
    dill.dump(matOfImages, file)

In [ ]:
with open('atl_183_1h_cluster.pkl', 'wb') as file:
    dill.dump(kmeans, file)

'''
#asd = cv2.imread("apple.png",cv2.IMREAD_UNCHANGED) 
cv2.imshow('image',asd)  
cv2.waitKey(0)
cv2.destroyAllWindows()

w*h*dpi*dpi

print ( kmeans.labels_ )
print ( kmeans.predict([[0, 0], [12, 3]]) )
print ( kmeans.cluster_centers_ )

region = 'ATL'
stormNo = 2
stormData = data[region][stormNo]
satellite = 'F17'
swath = 'S3'
freq = '150H'
imageData = stormData[satellite][swath][freq]
path = "..//..//MyCreatedData//"+ str(10) + "//" + region + "//" + str(stormNo) + "//" + satellite + "//" + swath + "//" + freq + "//"
'''